**1.Download S&P 500 Data**

In [4]:
import pandas as pd
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta as ta
import warnings
warnings.filterwarnings('ignore')



In [5]:
sp500=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date= '2024-09-30'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*10)
print(start_date)

2014-10-03 00:00:00


In [6]:
df= yf.download(tickers=symbols_list,start=start_date,end=end_date).stack()


[*********************100%***********************]  503 of 503 completed


working with all these huge rows and colums data would be inefficient so we would use df.stack()

In [7]:

df.index.names=['date','ticker']
df.columns=df.columns.str.lower()
df

Price                              adj close       close        high  \
date                      ticker                                       
2014-10-03 00:00:00+00:00 A        37.467007   40.708157   40.779686   
                          AAPL     22.113329   24.905001   25.052500   
                          ABBV     38.515564   58.689999   58.779999   
                          ABT      34.355877   41.790001   41.860001   
                          ACGL     18.469999   18.469999   18.490000   
...                                      ...         ...         ...   
2024-09-27 00:00:00+00:00 XYL     134.509995  134.509995  136.289993   
                          YUM     139.919998  139.919998  140.020004   
                          ZBH     107.980003  108.220001  110.190002   
                          ZBRA    368.600006  368.600006  374.029999   
                          ZTS     194.809998  194.809998  197.509995   

Price                                    low        open       volume  
date                      ticker                                       
2014-10-03 00:00:00+00:00 A        40.171673   40.221745    2205065.0  
                          AAPL     24.760000   24.860001  173878400.0  
                          ABBV     57.340000   57.419998   11675600.0  
                          ABT      41.279999   41.410000    3430900.0  
                          ACGL     18.320000   18.333332    1470300.0  
...                                      ...         ...          ...  
2024-09-27 00:00:00+00:00 XYL     133.779999  134.660004     691300.0  
                          YUM     136.990005  137.520004    2778300.0  
                          ZBH     108.190002  108.769997    1166200.0  
                          ZBRA    368.269989  372.299988     213600.0  
                          ZTS     194.419998  195.649994    1343700.0  

[1225998 rows x 6 columns]

<!-- # ### # ###### 2.CALCULATE FEATURES AND TECHNICAL INDICATORS FOR EACH STOCK
1**) garman-klass volatility
2)RSI
3)Bollinger bands
4)ATR
5)MACD
6)Dollar volume**
GK = sqrt((0.5 * (log(high / low))^2) - ((2 * log(2) - 1) * (log(close / open))^2))(garman klass simplified in code)

 -->


In [8]:
# **** GARMAN-KLASS VOLATILITY****

df['garman_klass_vol']=((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*(np.log(df['adj close'])-(np.log(df['open'])))**2
df

Price                              adj close       close        high  \
date                      ticker                                       
2014-10-03 00:00:00+00:00 A        37.467007   40.708157   40.779686   
                          AAPL     22.113329   24.905001   25.052500   
                          ABBV     38.515564   58.689999   58.779999   
                          ABT      34.355877   41.790001   41.860001   
                          ACGL     18.469999   18.469999   18.490000   
...                                      ...         ...         ...   
2024-09-27 00:00:00+00:00 XYL     134.509995  134.509995  136.289993   
                          YUM     139.919998  139.919998  140.020004   
                          ZBH     107.980003  108.220001  110.190002   
                          ZBRA    368.600006  368.600006  374.029999   
                          ZTS     194.809998  194.809998  197.509995   

Price                                    low        open       volume  \
date                      ticker                                        
2014-10-03 00:00:00+00:00 A        40.171673   40.221745    2205065.0   
                          AAPL     24.760000   24.860001  173878400.0   
                          ABBV     57.340000   57.419998   11675600.0   
                          ABT      41.279999   41.410000    3430900.0   
                          ACGL     18.320000   18.333332    1470300.0   
...                                      ...         ...          ...   
2024-09-27 00:00:00+00:00 XYL     133.779999  134.660004     691300.0   
                          YUM     136.990005  137.520004    2778300.0   
                          ZBH     108.190002  108.769997    1166200.0   
                          ZBRA    368.269989  372.299988     213600.0   
                          ZTS     194.419998  195.649994    1343700.0   

Price                             garman_klass_vol  
date                      ticker                    
2014-10-03 00:00:00+00:00 A              -0.001832  
                          AAPL           -0.005226  
                          ABBV           -0.061293  
                          ABT            -0.013375  
                          ACGL            0.000021  
...                                            ...  
2024-09-27 00:00:00+00:00 XYL             0.000172  
                          YUM             0.000124  
                          ZBH             0.000147  
                          ZBRA            0.000082  
                          ZTS             0.000117  

[1225998 rows x 7 columns]

In [9]:
#  RSI 
# A technical indicator that measures the speed and change of price movements of an asset,
#  such as a stock, index, or ETF. The RSI is displayed on a scale of 0 to 100, 
#  with readings above 70 indicating an overbought asset and readings below 30 indicating an oversold asset

df['rsi']=df.groupby(level=1)['adj close'].transform(lambda x : ta.rsi(close=x , length=20))
# df.xs('AAPL',level=1)['rsi'].plot()(#to ckeck the rsi yes its correct)

In [10]:
# bollinger bandsBollinger Bands are a technical analysis tool used 
# to measure market volatility and identify potential trading opportunities.
# They are made up of three lines: an upper band, a middle band, and a lower band. 
# The middle band is a moving average, while the upper and lower bands are based on standard deviation,
# which is a measure of price volatility.


# Check the output columns of the Bollinger Bands calculation
sample_output = ta.bbands(close=np.log1p(df['adj close']), length=20)
print(sample_output.columns)


Index(['BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0', 'BBB_20_2.0', 'BBP_20_2.0'], dtype='object')


In [11]:
# Calculate the lower Bollinger Band
df['bb_low'] = df.groupby(level=1)['adj close'].transform(
    lambda x: ta.bbands(close=np.log1p(x), length=20)['BBL_20_2.0'] if ta.bbands(close=np.log1p(x), length=20) is not None else np.nan
)

# Calculate the middle Bollinger Band
df['bb_middle'] = df.groupby(level=1)['adj close'].transform(
    lambda x: ta.bbands(close=np.log1p(x), length=20)['BBM_20_2.0'] if ta.bbands(close=np.log1p(x), length=20) is not None else np.nan
)

# Calculate the upper Bollinger Band
df['bb_high'] = df.groupby(level=1)['adj close'].transform(
    lambda x: ta.bbands(close=np.log1p(x), length=20)['BBU_20_2.0'] if ta.bbands(close=np.log1p(x), length=20) is not None else np.nan
)
df

Price                              adj close       close        high  \
date                      ticker                                       
2014-10-03 00:00:00+00:00 A        37.467007   40.708157   40.779686   
                          AAPL     22.113329   24.905001   25.052500   
                          ABBV     38.515564   58.689999   58.779999   
                          ABT      34.355877   41.790001   41.860001   
                          ACGL     18.469999   18.469999   18.490000   
...                                      ...         ...         ...   
2024-09-27 00:00:00+00:00 XYL     134.509995  134.509995  136.289993   
                          YUM     139.919998  139.919998  140.020004   
                          ZBH     107.980003  108.220001  110.190002   
                          ZBRA    368.600006  368.600006  374.029999   
                          ZTS     194.809998  194.809998  197.509995   

Price                                    low        open       volume  \
date                      ticker                                        
2014-10-03 00:00:00+00:00 A        40.171673   40.221745    2205065.0   
                          AAPL     24.760000   24.860001  173878400.0   
                          ABBV     57.340000   57.419998   11675600.0   
                          ABT      41.279999   41.410000    3430900.0   
                          ACGL     18.320000   18.333332    1470300.0   
...                                      ...         ...          ...   
2024-09-27 00:00:00+00:00 XYL     133.779999  134.660004     691300.0   
                          YUM     136.990005  137.520004    2778300.0   
                          ZBH     108.190002  108.769997    1166200.0   
                          ZBRA    368.269989  372.299988     213600.0   
                          ZTS     194.419998  195.649994    1343700.0   

Price                             garman_klass_vol        rsi    bb_low  \
date                      ticker                                          
2014-10-03 00:00:00+00:00 A              -0.001832        NaN       NaN   
                          AAPL           -0.005226        NaN       NaN   
                          ABBV           -0.061293        NaN       NaN   
                          ABT            -0.013375        NaN       NaN   
                          ACGL            0.000021        NaN       NaN   
...                                            ...        ...       ...   
2024-09-27 00:00:00+00:00 XYL             0.000172  52.810998  4.849547   
                          YUM             0.000124  62.536185  4.867805   
                          ZBH             0.000147  48.599348  4.623476   
                          ZBRA            0.000082  64.223784  5.759747   
                          ZTS             0.000117  61.511367  5.220925   

Price                             bb_middle   bb_high  
date                      ticker                       
2014-10-03 00:00:00+00:00 A             NaN       NaN  
                          AAPL          NaN       NaN  
                          ABBV          NaN       NaN  
                          ABT           NaN       NaN  
                          ACGL          NaN       NaN  
...                                     ...       ...  
2024-09-27 00:00:00+00:00 XYL      4.891114  4.932681  
                          YUM      4.902218  4.936632  
                          ZBH      4.685195  4.746914  
                          ZBRA     5.851378  5.943010  
                          ZTS      5.258852  5.296779  

[1225998 rows x 11 columns]

Average True Range (ATR) is a technical analysis tool that measures the volatility of a market by averaging the true ranges over a specified period: 
ATR is a key component of many traders' risk management strategies because it helps them:
Set stop losses
ATR helps traders set more accurate stop-loss levels that account for an asset's natural price fluctuations.
Determine position sizes
ATR helps traders determine position sizes that align with their risk tolerance.
Identify breakouts or trend reversals
Traders can monitor changes in ATR values to identify potential breakouts or trend reversals

In [12]:
#ATR
def compute_atr(stock_data):
    atr = ta.atr(high=stock_data['high'], low=stock_data['low'], close=stock_data['close'], length=14)
    # Check if ATR calculation is successful before proceeding
    if atr is not None:
        return atr.sub(atr.mean()).div(atr.std())
    else:
        return pd.Series([np.nan] * len(stock_data), index=stock_data.index)

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

#atr.sub(atr.mean()): Subtracts the mean of the ATR values from each ATR value, 
# centering the data around zero.
#.div(atr.std()): Divides each centered ATR value by the standard deviation of the ATR values, 
# scaling the data. This standardizes the ATR, which now has a mean of 0 and a standard deviation of 1.




**Moving Average Convergence Divergence (MACD) **is a technical indicator that helps investors identify entry points for buying or selling. It tracks the convergence and divergence of two exponential moving averages (EMAs) to gauge the strength of stock price movement: 
Calculation
The MACD line is calculated by subtracting the 26-period EMA from the 12-period EMA. 
Signal line
A nine-period EMA of the MACD line is plotted on the MACD chart as the signal line. 


In [13]:
#MACD
def compute_macd(close):
    # Calculate MACD
    macd = ta.macd(close=close, length=20)
    
    # Check if MACD calculation is successful
    if macd is not None and not macd.empty:
        macd_values = macd.iloc[:, 0]  # Get the MACD values
        return macd_values.sub(macd_values.mean()).div(macd_values.std())
    else:
        return pd.Series([np.nan] * len(close), index=close.index)

# Apply the MACD computation by grouping by the second level
df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)


In [81]:
df['dollar_volume']=(df['adj close']*df['volume'])/1e6
df

Price                              adj close       close        high  \
date                      ticker                                       
2014-10-03 00:00:00+00:00 A        37.467007   40.708157   40.779686   
                          AAPL     22.113329   24.905001   25.052500   
                          ABBV     38.515564   58.689999   58.779999   
                          ABT      34.355877   41.790001   41.860001   
                          ACGL     18.469999   18.469999   18.490000   
...                                      ...         ...         ...   
2024-09-27 00:00:00+00:00 XYL     134.509995  134.509995  136.289993   
                          YUM     139.919998  139.919998  140.020004   
                          ZBH     107.980003  108.220001  110.190002   
                          ZBRA    368.600006  368.600006  374.029999   
                          ZTS     194.809998  194.809998  197.509995   

Price                                    low        open       volume  \
date                      ticker                                        
2014-10-03 00:00:00+00:00 A        40.171673   40.221745    2205065.0   
                          AAPL     24.760000   24.860001  173878400.0   
                          ABBV     57.340000   57.419998   11675600.0   
                          ABT      41.279999   41.410000    3430900.0   
                          ACGL     18.320000   18.333332    1470300.0   
...                                      ...         ...          ...   
2024-09-27 00:00:00+00:00 XYL     133.779999  134.660004     691300.0   
                          YUM     136.990005  137.520004    2778300.0   
                          ZBH     108.190002  108.769997    1166200.0   
                          ZBRA    368.269989  372.299988     213600.0   
                          ZTS     194.419998  195.649994    1343700.0   

Price                             garman_klass_vol        rsi    bb_low  \
date                      ticker                                          
2014-10-03 00:00:00+00:00 A              -0.001832        NaN       NaN   
                          AAPL           -0.005226        NaN       NaN   
                          ABBV           -0.061293        NaN       NaN   
                          ABT            -0.013375        NaN       NaN   
                          ACGL            0.000021        NaN       NaN   
...                                            ...        ...       ...   
2024-09-27 00:00:00+00:00 XYL             0.000172  52.810998  4.849547   
                          YUM             0.000124  62.536185  4.867805   
                          ZBH             0.000147  48.599348  4.623476   
                          ZBRA            0.000082  64.223784  5.759747   
                          ZTS             0.000117  61.511367  5.220925   

Price                             bb_middle   bb_high       atr      macd  \
date                      ticker                                            
2014-10-03 00:00:00+00:00 A             NaN       NaN       NaN       NaN   
                          AAPL          NaN       NaN       NaN       NaN   
                          ABBV          NaN       NaN       NaN       NaN   
                          ABT           NaN       NaN       NaN       NaN   
                          ACGL          NaN       NaN       NaN       NaN   
...                                     ...       ...       ...       ...   
2024-09-27 00:00:00+00:00 XYL      4.891114  4.932681  0.935011  0.088067   
                          YUM      4.902218  4.936632  1.023418  0.131712   
                          ZBH      4.685195  4.746914 -0.623002 -0.377126   
                          ZBRA     5.851378  5.943010  0.350878  1.174472   
                          ZTS      5.258852  5.296779  0.774667  1.035597   

Price                             dollar_volume  
date                      ticker                 
2014-10-03 00:00:00+00:00 A        

### 3.Aggregate to monthly level and filter top 50 most liquid stocks for each month 

--to reduce training time and experiment with features and strategies , we convert the business-daily data to month-end frequency

df.unstack('ticker'):

This reshapes the DataFrame by "unstacking" the 'ticker' level of the index, which moves it from being a row index to being a column index.
This will produce a wide-format DataFrame where each unique 'ticker' value becomes a column header, and all other index levels are maintained.
['dollar_volume']:

After unstacking, this selects the 'dollar_volume' column (or MultiIndex level), focusing only on this column for further operations.

.resample('M'):
This resamples the data to a monthly frequency (using 'M' for monthly).
The resample method is typically used for time series data and will create a new index with the first date of each month.

.mean():
This calculates the mean of 'dollar_volume' for each ticker within each monthly period.
The result is a DataFrame with a monthly time index and each ticker as a separate column, where the values are the mean of 'dollar_volume' for each ticker in each month.

.stack('ticker'):
This "stacks" the ticker level back from being a column index to being part of the row index, returning to a long-format DataFrame.
After stacking, the DataFrame will have a MultiIndex with date (monthly) and ticker levels.

.to_frame('dollar_volume'):
Converts the Series back to a DataFrame and renames the column to 'dollar_volume'.
This step is often used when a single-column DataFrame is preferred for consistent structure or further manipulation.

In [82]:
df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume')

dollar_volume
date                      ticker               
2014-10-31 00:00:00+00:00 A          129.925499
                          AAPL      5410.345808
                          ABBV       790.715507
                          ABT        195.698921
                          ACGL        23.590734
...                                         ...
2024-09-30 00:00:00+00:00 XYL        160.566119
                          YUM        287.368688
                          ZBH        192.681882
                          ZBRA       121.973536
                          ZTS        369.222766

[58555 rows x 1 columns]

In [83]:
# technical indicators column:

# last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume','volume','open','close','high','low']]
# last_cols


#now we would concetanate the dollar volume with rest of the indicators:
data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
           df.unstack()[last_cols].resample('M').last().stack('ticker')],axis=1)).dropna()

data

dollar_volume   adj close  garman_klass_vol  \
date                      ticker                                                
2014-11-30 00:00:00+00:00 A          112.789832   39.337070         -0.002615   
                          AAPL      4371.096444   26.514177         -0.005261   
                          ABBV       273.920929   45.751591         -0.065179   
                          ABT        152.154593   36.786419         -0.013803   
                          ACGL        27.891787   19.106667          0.000021   
...                                         ...         ...               ...   
2024-09-30 00:00:00+00:00 XYL        160.566119  134.509995          0.000172   
                          YUM        287.368688  139.919998          0.000124   
                          ZBH        192.681882  107.980003          0.000147   
                          ZBRA       121.973536  368.600006          0.000082   
                          ZTS        369.222766  194.809998          0.000117   

                                        rsi    bb_low  bb_middle   bb_high  \
date                      ticker                                             
2014-11-30 00:00:00+00:00 A       65.857372  3.624898   3.665358  3.705817   
                          AAPL    79.936324  3.196737   3.264268  3.331800   
                          ABBV    74.712797  3.715873   3.781591  3.847309   
                          ABT     62.980215  3.601911   3.618318  3.634725   
                          ACGL    59.646062  2.986044   2.997348  3.008653   
...                                     ...       ...        ...       ...   
2024-09-30 00:00:00+00:00 XYL     52.810998  4.849547   4.891114  4.932681   
                          YUM     62.536185  4.867805   4.902218  4.936632   
                          ZBH     48.599348  4.623476   4.685195  4.746914   
                          ZBRA    64.223784  5.759747   5.851378  5.943010   
                          ZTS     61.511367  5.220925   5.258852  5.296779   

                                       atr      macd  
date                      ticker                      
2014-11-30 00:00:00+00:00 A      -1.121415  0.346491  
                          AAPL   -1.022122  0.198583  
                          ABBV   -1.075526  0.864664  
                          ABT    -1.148236  0.241694  
                          ACGL   -1.227769 -0.112052  
...                                    ...       ...  
2024-09-30 00:00:00+00:00 XYL     0.935011  0.088067  
                          YUM     1.023418  0.131712  
                          ZBH    -0.623002 -0.377126  
                          ZBRA    0.350878  1.174472  
                          ZTS     0.774667  1.035597  

[58048 rows x 9 columns]

**calculate 5 year rolling average of dollar volume for each stocks before filtering**

In [84]:
data['dollar_volume']=(data['dollar_volume'].unstack('ticker').rolling(5*12).mean().stack())

data['dollar_vol_rank']=(data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150]

data

#we have calculate the monthly values for all the features 


dollar_volume   adj close  garman_klass_vol  \
date                      ticker                                                
2019-10-31 00:00:00+00:00 AAPL      4870.537642   60.177776          0.000923   
                          ABBV       410.910317   64.195107         -0.018022   
                          ABT        305.309014   76.628838         -0.002921   
                          ACN        263.728002  172.285385         -0.002118   
                          ADBE       457.344572  277.929993          0.000104   
...                                         ...         ...               ...   
2024-09-30 00:00:00+00:00 VZ         776.954034   44.200008          0.000025   
                          WFC        995.762963   55.900002          0.000043   
                          WMT       1037.630360   79.779999          0.000036   
                          XOM       1538.960971  115.820000          0.000182   
                          ZTS        337.872609  194.809998          0.000117   

                                        rsi    bb_low  bb_middle   bb_high  \
date                      ticker                                             
2019-10-31 00:00:00+00:00 AAPL    68.908180  4.006348   4.067200  4.128052   
                          ABBV    70.480961  4.068833   4.129044  4.189254   
                          ABT     54.403523  4.288739   4.325429  4.362118   
                          ACN     47.064243  5.132833   5.152503  5.172172   
                          ADBE    51.985924  5.570602   5.610361  5.650119   
...                                     ...       ...        ...       ...   
2024-09-30 00:00:00+00:00 VZ      65.041479  3.726660   3.781470  3.836280   
                          WFC     51.187900  3.961944   4.026840  4.091735   
                          WMT     63.358678  4.345042   4.380543  4.416044   
                          XOM     51.821142  4.706781   4.745506  4.784230   
                          ZTS     61.511367  5.220925   5.258852  5.296779   

                                       atr      macd  dollar_vol_rank  
date                      ticker                                       
2019-10-31 00:00:00+00:00 AAPL   -0.592924  0.515936              1.0  
                          ABBV   -0.941640  0.762890             58.0  
                          ABT    -0.009821  0.041810             83.0  
                          ACN    -0.348493 -0.500079            101.0  
                          ADBE   -0.387039 -0.302571             49.0  
...                                    ...       ...              ...  
2024-09-30 00:00:00+00:00 VZ     -0.326941  1.906847             53.0  
                          WFC     1.731254 -0.238594             37.0  
                          WMT     2.010067  2.928695             35.0  
                          XOM     1.213081 -0.132851             16.0  
                          ZTS     0.774667  1.035597            144.0  

[8940 rows x 10 columns]

**4.calculate monthly returns for different time horizons as features**

-- To capture time series dynamics that reflect ,for example momentum patterns ,we compute historical returns using method .pct_change(lag), that is returns over various monthly periods as indentified by lags.

In [88]:
def calculate_returns(df):
    outlier_cutoff=0.005
    lags=[1,2,3,6,9,12]
    
    
    
    for lag in lags:
        
        df[f'%return_{lag}m']=(df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x : x.clip(lower=x.quantile(outlier_cutoff),upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df

# .pipe(lambda x: ...): This is a functional approach to modify the data.
# It allows you to apply a function (in this case, a lambda function) to the Series.
# x.clip(...): Clipping limits the values of the Series. The lower and upper parameters specify the bounds.
# lower=x.quantile(outlier_cutoff): The lower bound is set to the value at the specified quantile, 
# defined by outlier_cutoff. This means any values below this quantile will be replaced with this value.
# upper=x.quantile(1-outlier_cutoff): The upper bound is similarly set to the value at the upper quantile.

data = data.groupby(level=1).apply(calculate_returns).dropna()
data

dollar_volume  \
date                      ticker date                      ticker date                      ticker                  
2021-10-31 00:00:00+00:00 C      2021-10-31 00:00:00+00:00 C      2021-10-31 00:00:00+00:00 C          971.400990   
                          CAT    2021-10-31 00:00:00+00:00 CAT    2021-10-31 00:00:00+00:00 CAT        532.534439   
                          CHTR   2021-10-31 00:00:00+00:00 CHTR   2021-10-31 00:00:00+00:00 CHTR       563.025342   
                          CI     2021-10-31 00:00:00+00:00 CI     2021-10-31 00:00:00+00:00 CI         350.301335   
                          CMCSA  2021-10-31 00:00:00+00:00 CMCSA  2021-10-31 00:00:00+00:00 CMCSA      737.873017   
...                                                                                                           ...   
2024-09-30 00:00:00+00:00 VRTX   2024-09-30 00:00:00+00:00 VRTX   2024-09-30 00:00:00+00:00 VRTX       432.978800   
                          VZ     2024-09-30 00:00:00+00:00 VZ     2024-09-30 00:00:00+00:00 VZ         776.954034   
                          WFC    2024-09-30 00:00:00+00:00 WFC    2024-09-30 00:00:00+00:00 WFC        995.762963   
                          WMT    2024-09-30 00:00:00+00:00 WMT    2024-09-30 00:00:00+00:00 WMT       1037.630360   
                          XOM    2024-09-30 00:00:00+00:00 XOM    2024-09-30 00:00:00+00:00 XOM       1538.960971   

                                                                                                     adj close  \
date                      ticker date                      ticker date                      ticker               
2021-10-31 00:00:00+00:00 C      2021-10-31 00:00:00+00:00 C      2021-10-31 00:00:00+00:00 C        61.814140   
                          CAT    2021-10-31 00:00:00+00:00 CAT    2021-10-31 00:00:00+00:00 CAT     192.242569   
                          CHTR   2021-10-31 00:00:00+00:00 CHTR   2021-10-31 00:00:00+00:00 CHTR    674.890015   
                          CI     2021-10-31 00:00:00+00:00 CI     2021-10-31 00:00:00+00:00 CI      202.860687   
                          CMCSA  2021-10-31 00:00:00+00:00 CMCSA  2021-10-31 00:00:00+00:00 CMCSA    47.252708   
...                                                                                                        ...   
2024-09-30 00:00:00+00:00 VRTX   2024-09-30 00:00:00+00:00 VRTX   2024-09-30 00:00:00+00:00 VRTX    462.880005   
                          VZ     2024-09-30 00:00:00+00:00 VZ     2024-09-30 00:00:00+00:00 VZ       44.200008   
                          WFC    2024-09-30 00:00:00+00:00 WFC    2024-09-30 00:00:00+00:00 WFC      55.900002   
                          WMT    2024-09-30 00:00:00+00:00 WMT    2024-09-30 00:00:00+00:00 WMT      79.779999   
                          XOM    2024-09-30 00:00:00+00:00 XOM    2024-09-30 00:00:00+00:00 XOM     115.820000   

                                                                                                    garman_klass_vol  \
date                      ticker date                      ticker date                      ticker                     
2021-10-31 00:00:00+00:00 C      2021-10-31 00:00:00+00:00 C      2021-10-31 00:00:00+00:00 C              -0.006373   
                          CAT    2021-10-31 00:00:00+00:00 CAT    2021-10-31 00:00:00+00:00 CAT            -0.001898   
                          CHTR   2021-10-31 00:00:00+00:00 CHTR   2021-10-31 00:00:00+00:00 CHTR            0.000779   
                          CI     2021-10-31 00:00:00+00:00 CI     2021-10-31 00:00:00+00:00 CI             -0.001215   
                          CMCSA  2021-10-31 00:00:00+00:00 CMCSA  2021-10-31 00:00:00+00:00 CMCSA          -0.002528   
...                                                                                                              ...   
2024-09-30 00:00:00+00:00 VRTX   2024-09-30 00:00:00+00:00 VRTX   2024-09-30 00:00:00+00:00 VRTX            0.000068   
               

5. Download fama-french factors and calculate rolling factor betas

--we will introduce fema french factors

Index([], dtype='object', name='ticker')
